# 증강할 거... 메신저피싱형, 택배사칭형, 가족지인사칭형
메신저피싱형 252건 - 168건
택배사칭형 199건 - 161건
가족지인사칭형 176건 -125건

In [1]:
import pandas as pd
import os
import time
import re
from dotenv import load_dotenv
from openai import OpenAI

#.env에서 OpenAI API 키 불러오기
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("❌ .env에 OPENAI_API_KEY가 설정되지 않았습니다.")

client = OpenAI(api_key=openai_api_key)

In [13]:
import pandas as pd
import os
import time
from openai import OpenAI

# OpenAI API 설정
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 역번역 함수 (문장 단위)
def gpt_back_translate_en(text):
    try:
        print(f"역번역 시작: {text[:30]}...")
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "너는 한국 보이스피싱 탐지 시스템 개발 프로젝트에 참여 중인 데이터 증강 전문가야. "
                        "사용자가 주는 한국어 문장은 보이스피싱 사례 텍스트야. "
                        "너의 임무는 이 문장을 먼저 영어로 자연스럽게 번역한 뒤, 다시 한국어로 자연스럽고 다양하게 재구성해주는 거야. "
                        "단, 원래 문장의 의미와 맥락은 유지하되 표현을 바꾸고, 자연스럽고 실제 통화처럼 들리도록 만들어야 해. "
                        "**최종 출력은 번역된 한국어 문장만 보여줘.** 최종 결과에 영어가 나오면 실패. 영어 텍스트 삭제"
                    ),
                },
                {
                    "role": "user",
                    "content": f"다음 문장을 영어로 번역한 후 다시 한국어로 자연스럽게 재번역해줘. 최종출력은 한국어 문장만:\n\n'{text}'",
                },
            ],
            temperature=0.9,
            max_tokens=500,
        )
        result = response.choices[0].message.content.strip()
        print(f"역번역 완료: {result[:30]}...")
        return result
    except Exception as e:
        print(f"[오류] 역번역 실패: {e}")
        return None

# 경로 및 타겟 유형 설정
INPUT_FILE = "C:/Users/user/Downloads/0708/woogawooga_project/dataset/phishing_total.csv"
OUTPUT_FILE = "back_translated.csv"
FAILED_FILE = "failed_log.csv"
TARGET_TYPES = ['메신저피싱형', '택배사칭형', '가족지인사칭형']

# 처리된 file_name 불러오기
def load_processed_file_names():
    if not os.path.exists(OUTPUT_FILE):
        print(" 처리된 파일 없음 (새 시작)")
        return set()
    try:
        df_done = pd.read_csv(OUTPUT_FILE)
        if 'file_name' not in df_done.columns:
            print("'file_name' 컬럼이 없음")
            return set()
        processed = set(df_done['file_name'].dropna().unique())
        print(f"처리된 file_name 수: {len(processed)}개")
        return processed
    except Exception as e:
        print(f" 처리된 목록 로딩 실패: {e}")
        return set()

# CSV 헤더 생성 (없을 경우)
if not os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as f:
        f.write("file_name,speaker,phishing_type,text,back_translated\n")

if not os.path.exists(FAILED_FILE):
    with open(FAILED_FILE, 'w', encoding='utf-8', newline='') as f:
        f.write("file_name,speaker,phishing_type,text,error_type\n")

# 전체 데이터 로딩 및 필터링
df = pd.read_csv(INPUT_FILE)
df = df[df['phishing_type'].isin(TARGET_TYPES)]
grouped = df.groupby("file_name")

processed_file_names = load_processed_file_names()

# 메인 처리 루프
for file_name, group in grouped:
    if file_name in processed_file_names:
        print(f"건너뜀 (이미 처리됨): {file_name}")
        continue

    speaker_list = group['speaker'].tolist()
    text_list = group['text'].tolist()
    phishing_type = group['phishing_type'].iloc[0]

    for spk, text in zip(speaker_list, text_list):
        text = str(text).strip()
        translated = gpt_back_translate_en(text)

        if translated is None:
            with open(FAILED_FILE, 'a', encoding='utf-8') as f_fail:
                f_fail.write(f"{file_name},{spk},{phishing_type},{text},gpt_fail\n")
            translated = ""  # 일관성 위해 빈 값 처리

        text_clean = text.replace("\n", " ").replace(",", " ")
        translated_clean = translated.replace("\n", " ").replace(",", " ")

        with open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
            f_out.write(f"{file_name},{spk},{phishing_type},{text_clean},{translated_clean}\n")

        time.sleep(1.0)  # API 과부하 방지

    print(f" 처리 완료: {file_name}")
    processed_file_names.add(file_name)  # 처리 목록에 수동 추가


처리된 file_name 수: 275개
건너뜀 (이미 처리됨): phishing_000
건너뜀 (이미 처리됨): phishing_001
건너뜀 (이미 처리됨): phishing_002
건너뜀 (이미 처리됨): phishing_003
건너뜀 (이미 처리됨): phishing_004
건너뜀 (이미 처리됨): phishing_005
건너뜀 (이미 처리됨): phishing_006
건너뜀 (이미 처리됨): phishing_007
건너뜀 (이미 처리됨): phishing_008
건너뜀 (이미 처리됨): phishing_009
건너뜀 (이미 처리됨): phishing_010
건너뜀 (이미 처리됨): phishing_011
건너뜀 (이미 처리됨): phishing_012
건너뜀 (이미 처리됨): phishing_013
건너뜀 (이미 처리됨): phishing_014
건너뜀 (이미 처리됨): phishing_015
건너뜀 (이미 처리됨): phishing_016
건너뜀 (이미 처리됨): phishing_017
건너뜀 (이미 처리됨): phishing_018
건너뜀 (이미 처리됨): phishing_019
건너뜀 (이미 처리됨): phishing_020
건너뜀 (이미 처리됨): phishing_021
건너뜀 (이미 처리됨): phishing_022
건너뜀 (이미 처리됨): phishing_023
건너뜀 (이미 처리됨): phishing_024
건너뜀 (이미 처리됨): phishing_025
건너뜀 (이미 처리됨): phishing_026
건너뜀 (이미 처리됨): phishing_027
건너뜀 (이미 처리됨): phishing_028
건너뜀 (이미 처리됨): phishing_029
건너뜀 (이미 처리됨): phishing_030
건너뜀 (이미 처리됨): phishing_031
건너뜀 (이미 처리됨): phishing_032
건너뜀 (이미 처리됨): phishing_033
건너뜀 (이미 처리됨): phishing_034
건너뜀 (이미 처리됨): phishing_035
건너뜀 (이

In [15]:
df=pd.read_csv('C:/Users/user/Downloads/0708/woogawooga_project/dataset_create/Chaeyeon/back_translated1cy.csv')
# 'file_name' 뒤에 '_cy1' 붙이기
df['file_name'] = df['file_name'].astype(str) + '_cy1'

# 새 파일로 저장
df.to_csv("back_translated_cy1.csv", index=False, encoding='utf-8')

# 남은 거...
메신저피싱형 84건
택배사칭형 38건
가족지인사칭형 51건 

In [18]:
import pandas as pd
import random
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

# ✅ 1. 환경 설정
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

# ✅ 2. GPT-4o 역번역 함수 (한→중→한)
def gpt_back_translate_zh(text):
    try:
        print(f"🔁 역번역 시작: {text[:30]}...")
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "너는 한국 보이스피싱 탐지 프로젝트의 데이터 증강 전문가야. "
                        "사용자가 제공한 한국어 문장을 먼저 중국어로 번역한 다음, 다시 자연스러운 한국어로 재번역해줘. "
                        "표현은 바꾸되 의미와 맥락은 유지하고, 실제 통화처럼 들리게 해. **최종 출력은 반드시 한국어 문장**."
                    ),
                },
                {
                    "role": "user",
                    "content": f"다음 문장을 중국어로 번역한 뒤, 다시 한국어로 자연스럽게 재번역해줘. 출력은 한국어 문장만:\n\n'{text}'",
                },
            ],
            temperature=1.0,
            max_tokens=500,
        )
        result = response.choices[0].message.content.strip()
        print(f"✅ 역번역 완료: {result[:30]}...")
        return result
    except Exception as e:
        print(f"[❌ 오류] 역번역 실패: {e}")
        return None

# ✅ 3. 파일 경로 및 설정
INPUT_FILE = "back_translated_cy1.csv"
OUTPUT_FILE = "back_translated_cy3.csv"
FAILED_LOG = "cy3_failed_log.csv"

augmentation_targets = {
    '메신저피싱형': 84,
    '택배사칭형': 38,
    '가족지인사칭형': 51,
}

# ✅ 4. 기존 데이터 불러오기
df = pd.read_csv(INPUT_FILE)
df['base_file_name'] = df['file_name'].str.replace(r'_cy\d+$', '', regex=True)

# ✅ 5. 파일 초기화
if not os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        f.write("file_name,speaker,phishing_type,text,back_translated\n")

if not os.path.exists(FAILED_LOG):
    with open(FAILED_LOG, 'w', encoding='utf-8') as f:
        f.write("file_name,speaker,phishing_type,text,error_type\n")

# ✅ 6. 증강 실행
for phishing_type, n_needed in augmentation_targets.items():
    print(f"\n📌 {phishing_type} - {n_needed}건 증강 시작")
    subset = df[df['phishing_type'] == phishing_type]
    file_names = subset['base_file_name'].unique().tolist()
    sampled_file_names = random.choices(file_names, k=n_needed)

    for i, fname in enumerate(sampled_file_names):
        group = subset[subset['base_file_name'] == fname].copy()
        new_file_name = f"{fname}_cy3_{i}"

        for _, row in group.iterrows():
            spk = row['speaker']
            text = str(row['text']).strip()
            phishing_type = row['phishing_type']
            translated = gpt_back_translate_zh(text)

            if translated is None:
                with open(FAILED_LOG, 'a', encoding='utf-8') as f_log:
                    f_log.write(f"{new_file_name},{spk},{phishing_type},{text},gpt_fail\n")
                translated = ""

            # 저장
            text_clean = text.replace(",", " ").replace("\n", " ")
            translated_clean = translated.replace(",", " ").replace("\n", " ")
            with open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out:
                f_out.write(f"{new_file_name},{spk},{phishing_type},{text_clean},{translated_clean}\n")

            time.sleep(1.0)

print("\n✅ 전체 재역번역 완료! 저장됨 →", OUTPUT_FILE)



📌 메신저피싱형 - 84건 증강 시작
🔁 역번역 시작: 여보세요  4시 되시니까 몰라....
✅ 역번역 완료: 여보세요, 4시가 됐는지 잘 모르겠어요....
🔁 역번역 시작: 네  안녕하세요  서울중앙지금이고요  저는 천담범대 수...
✅ 역번역 완료: 네, 안녕하세요. 저는 서울중앙지검 천담범대 수사팀의 ...
🔁 역번역 시작: OOO님  안녕하세요. 친구가 메신저로 부탁드리는 메시...
✅ 역번역 완료: OOO님 안녕하세요. 친구가 메신저로 부탁할 메시지를 ...
🔁 역번역 시작: 안녕하세요  무슨 부탁이죠?...
✅ 역번역 완료: 안녕하세요, 어떤 도움이 필요하신가요?...
🔁 역번역 시작: 급하게 돈이 필요해서 빌려주실 수 있는지 여쭤보려 합니...
✅ 역번역 완료: 급하게 돈이 필요해서 부탁드릴 게 있어서 연락드렸어요....
🔁 역번역 시작: 얼마나 필요한가요? 그리고 왜 직접 전화하지 않나요?...
✅ 역번역 완료: 얼마나 필요하세요? 그리고 직접 전화는 왜 안 하시는 ...
🔁 역번역 시작: 지금 해외라 통화가 어려워서요  50만원이 필요합니다....
✅ 역번역 완료: 지금 해외에 있어서 통화가 힘들고, 50만 원이 필요해...
🔁 역번역 시작: 저는 돈 거래는 직접 만나서만 해요. 연락해보겠습니다....
✅ 역번역 완료: 저는 돈 거래는 꼭 직접 만나서만 해요. 연락해볼게요....
🔁 역번역 시작: 지금 바로 만나기는 어려워요. 계좌 이체를 부탁드려요....
✅ 역번역 완료: 지금 바로 만나는 게 어렵습니다. 계좌로 송금해 주시겠...
🔁 역번역 시작: 그렇다면 좀 곤란하네요. 직접 친구랑 통화해볼게요....
✅ 역번역 완료: 그럼 좀 어려울 것 같아요. 친구한테 직접 전화해볼게요...
🔁 역번역 시작: OOO님  메신저 보안 팀입니다. 계정이 해킹된 흔적이...
✅ 역번역 완료: OOO님, 메신저 보안팀입니다. 고객님의 계정에서 해킹...
🔁 역번역 시작: 어떻게 해야 하나요? 좀 도와주세요....
✅ 역번역 완료: 어떻게 해야 할지 모르겠어